In [10]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/MyDrive/Colab'

Mounted at /content/drive/
/content/drive/MyDrive/Colab


In [6]:
!pip install Keras-Preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.7 MB/s eta 0:00:00


# Simple Test (Create Model Json File and Load)

In [1]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dropout, BatchNormalization, Dense, Embedding

# Define model parameters
vocab_size = 10000   # Vocabulary size
embedding_dim = 128  # Word embedding dimension
input_length = 100   # Max sequence length
num_classes = 5      # Number of output classes

# Build the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_length),  # Embedding layer
    Bidirectional(LSTM(64, return_sequences=True)),  # BiLSTM layer (with sequence output)
    Dropout(0.5),
    BatchNormalization(),  # Batch Normalization
    Bidirectional(LSTM(32, return_sequences=False)),  # Another BiLSTM layer
    Dropout(0.3),
    BatchNormalization(),
    Dense(num_classes, activation='softmax')  # Output layer for classification
])

# Compile the model
model.build((256, None))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Load Model from JSON
config_model = model.to_json()
loaded_model = keras.models.model_from_json(config_model)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (256, None, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (256, None, 128)            │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (256, None, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (256, None, 128)            │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (256, 64)                   │          41,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (256, 64)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (256, 64)                   │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (256, 5)                    │             325 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,421,125 (5.42 MB)

 Trainable params: 1,420,741 (5.42 MB)

 Non-trainable params: 384 (1.50 KB)

In [2]:
config_model

'{"module": "keras", "class_name": "Sequential", "config": {"name": "sequential", "trainable": true, "dtype": {"module": "keras", "class_name": "DTypePolicy", "config": {"name": "float32"}, "registered_name": null}, "layers": [{"module": "keras.layers", "class_name": "InputLayer", "config": {"batch_shape": [256, null], "dtype": "float32", "sparse": false, "name": "input_layer"}, "registered_name": null}, {"module": "keras.layers", "class_name": "Embedding", "config": {"name": "embedding", "trainable": true, "dtype": {"module": "keras", "class_name": "DTypePolicy", "config": {"name": "float32"}, "registered_name": null}, "input_dim": 10000, "output_dim": 128, "embeddings_initializer": {"module": "keras.initializers", "class_name": "RandomUniform", "config": {"seed": null, "minval": -0.05, "maxval": 0.05}, "registered_name": null}, "embeddings_regularizer": null, "activity_regularizer": null, "embeddings_constraint": null, "mask_zero": false}, "registered_name": null, "build_config": {"i

# Regenerate Model Json Files (Easy Solution)

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, BatchNormalization, Dense, Layer
from tensorflow.keras.models import Model
import keras
from keras import initializers, regularizers, constraints
import numpy as np

In [6]:
@keras.saving.register_keras_serializable()
class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        #self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = 256
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
    #print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        #return input_shape[0], input_shape[-1]
        return input_shape[0],  self.features_dim

    # Add new
    @classmethod
    def from_config(cls,config):
        return cls(**config)

In [7]:
# Define model parameters
max_review_length = 256
vocab_size = 99  # Vocabulary size for embedding
embedding_dim = 300  # Embedding vector size
input_length = max_review_length  # Maximum sequence length
num_classes = 2199  # Number of output classes
(embedding_matrix_shape0, embedding_matrix_shape1) = (99, 300)
embedding_matrix = np.random.rand(embedding_matrix_shape0, embedding_matrix_shape1)

inputs=Input(shape=(max_review_length,), dtype="int32")
embedding=Embedding(embedding_matrix_shape0, embedding_matrix_shape1,
                    weights=[embedding_matrix], input_length=max_review_length)(inputs)
lstm_out=Bidirectional(LSTM(128, return_sequences=True))(embedding)
attention=Attention()(lstm_out)
attention=Dense(256, activation="relu")(attention)
attention=BatchNormalization()(attention)
output=Dense(2199, activation='softmax')(attention)
model=Model(inputs=inputs, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Load Model
config_model = model.to_json()
loaded_model = keras.models.model_from_json(config_model, custom_objects={'Attention': Attention()})

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 256, 300)            │          29,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 256, 256)            │         439,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention (Attention)                │ (None, 256)                 │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2199)                │         565,143 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,101,467 (4.20 MB)

 Trainable params: 1,100,955 (4.20 MB)

 Non-trainable params: 512 (2.00 KB)

In [8]:
config_model

'{"module": "keras.src.models.functional", "class_name": "Functional", "config": {"name": "functional_9", "trainable": true, "layers": [{"module": "keras.layers", "class_name": "InputLayer", "config": {"batch_shape": [null, 256], "dtype": "int32", "sparse": false, "name": "input_layer_2"}, "registered_name": null, "name": "input_layer_2", "inbound_nodes": []}, {"module": "keras.layers", "class_name": "Embedding", "config": {"name": "embedding_2", "trainable": true, "dtype": {"module": "keras", "class_name": "DTypePolicy", "config": {"name": "float32"}, "registered_name": null}, "input_dim": 99, "output_dim": 300, "embeddings_initializer": {"module": "keras.initializers", "class_name": "RandomUniform", "config": {"seed": null, "minval": -0.05, "maxval": 0.05}, "registered_name": null}, "embeddings_regularizer": null, "activity_regularizer": null, "embeddings_constraint": null, "mask_zero": false}, "registered_name": null, "build_config": {"shapes_dict": {}}, "name": "embedding_2", "inbo

# Complicated Solution

## To Check the Weights Available in Model Weights File

In [12]:
import h5py

def print_h5_structure(name, obj):
    if isinstance(obj, h5py.Dataset):
        print(f"Dataset: {name}, Shape: {obj.shape}, Type: {obj.dtype}")
    elif isinstance(obj, h5py.Group):
        print(f"Group: {name}")

with h5py.File('char_attn_lstm_model.h5', 'r') as f:
    f.visititems(print_h5_structure)

Group: attention_20
Group: attention_20/attention_20
Dataset: attention_20/attention_20/attention_20_W:0, Shape: (512,), Type: float32
Dataset: attention_20/attention_20/attention_20_b:0, Shape: (256,), Type: float32
Group: batch_normalization_59
Group: batch_normalization_59/batch_normalization_59
Dataset: batch_normalization_59/batch_normalization_59/beta:0, Shape: (1024,), Type: float32
Dataset: batch_normalization_59/batch_normalization_59/gamma:0, Shape: (1024,), Type: float32
Dataset: batch_normalization_59/batch_normalization_59/moving_mean:0, Shape: (1024,), Type: float32
Dataset: batch_normalization_59/batch_normalization_59/moving_variance:0, Shape: (1024,), Type: float32
Group: bidirectional_21
Group: bidirectional_21/bidirectional_21
Group: bidirectional_21/bidirectional_21/backward_lstm_21
Dataset: bidirectional_21/bidirectional_21/backward_lstm_21/bias:0, Shape: (1024,), Type: float32
Dataset: bidirectional_21/bidirectional_21/backward_lstm_21/kernel:0, Shape: (300, 1024)

## To Check the Weights Required in Model Json Files

In [21]:
config_model = model.to_json()
tmp_model = keras.models.model_from_json(config_model, custom_objects={'Attention': Attention()})

for layer in tmp_model.layers:
    print(layer.name)
    print([w.shape for w in tmp_model.get_layer(layer.name).get_weights()])

input_layer_27
[]
embedding_60
[(99, 300)]
bidirectional_21
[(300, 1024), (256, 1024), (1024,), (300, 1024), (256, 1024), (1024,)]
attention_20
[(512,), (256,)]
dense_118
[(512, 1024), (1024,)]
batch_normalization_59
[(1024,), (1024,), (1024,), (1024,)]
dense_119
[(1024, 2199), (2199,)]


## Assign Weights to Model

In [23]:
from keras.models import model_from_json
# json_file = open("char_attn_lstm_model_upd2025.json","r")
json_file = open("test2.json","r")
loaded_model_json = json_file.read()
json_file.close()

tmp_model = model_from_json(loaded_model_json, custom_objects={'Attention': Attention()})

In [14]:
import h5py
with h5py.File('char_attn_lstm_model.h5', 'r') as f:
    tmp_dict = {
        'attention_20_W': f['attention_20']['attention_20']['attention_20_W:0'][()],
        'attention_20_b': f['attention_20']['attention_20']['attention_20_b:0'][()],
        'batch_normalization_59_beta': f['batch_normalization_59']['batch_normalization_59']['beta:0'][()],
        'batch_normalization_59_gamma': f['batch_normalization_59']['batch_normalization_59']['gamma:0'][()],
        'batch_normalization_59_moving_mean': f['batch_normalization_59']['batch_normalization_59']['moving_mean:0'][()],
        'batch_normalization_59_moving_variance': f['batch_normalization_59']['batch_normalization_59']['moving_variance:0'][()],
        'backward_lstm_21_bias': f['bidirectional_21']['bidirectional_21']['backward_lstm_21']['bias:0'][()],
        'backward_lstm_21_kernel': f['bidirectional_21']['bidirectional_21']['backward_lstm_21']['kernel:0'][()],
        'backward_lstm_21_recurrent_kernel': f['bidirectional_21']['bidirectional_21']['backward_lstm_21']['recurrent_kernel:0'][()],
        'forward_lstm_21_bias': f['bidirectional_21']['bidirectional_21']['forward_lstm_21']['bias:0'][()],
        'forward_lstm_21_kernel': f['bidirectional_21']['bidirectional_21']['forward_lstm_21']['kernel:0'][()],
        'forward_lstm_21_recurrent_kernel': f['bidirectional_21']['bidirectional_21']['forward_lstm_21']['recurrent_kernel:0'][()],
        'dense_118_bias': f['dense_118']['dense_118']['bias:0'][()],
        'dense_118_kernel': f['dense_118']['dense_118']['kernel:0'][()],
        'dense_119_bias': f['dense_119']['dense_119']['bias:0'][()],
        'dense_119_kernel': f['dense_119']['dense_119']['kernel:0'][()],
        'embedding_60': f['embedding_60']['embedding_60']['embeddings:0'][()],
        }

In [24]:
# Set dense_119
tmp_model.get_layer('dense_119').set_weights([tmp_dict['dense_119_kernel'], tmp_dict['dense_119_bias']])
# Set dense_118
tmp_model.get_layer('dense_118').set_weights([tmp_dict['dense_118_kernel'], tmp_dict['dense_118_bias']])
# Embedding:
tmp_model.get_layer('embedding_60').set_weights([tmp_dict['embedding_60']])
# bidirectional_21
tmp_model.get_layer('bidirectional_21').set_weights([
    tmp_dict['forward_lstm_21_kernel'], tmp_dict['forward_lstm_21_recurrent_kernel'],
    tmp_dict['forward_lstm_21_bias'], tmp_dict['backward_lstm_21_kernel'],
    tmp_dict['backward_lstm_21_recurrent_kernel'],
    tmp_dict['backward_lstm_21_bias']])
# attention_20
tmp_model.get_layer('attention_20').set_weights([tmp_dict['attention_20_W'], tmp_dict['attention_20_b']])

# batch_normalization_59
tmp_model.get_layer('batch_normalization_59').set_weights([
    tmp_dict['batch_normalization_59_beta'], tmp_dict['batch_normalization_59_gamma'],
    tmp_dict['batch_normalization_59_moving_mean'], tmp_dict['batch_normalization_59_moving_variance']])